In [1]:
import os

In [2]:
!pwd

/home/jovyan/notebooks/analysis/experiment_2


In [3]:
import sys
sys.path.append('/home/jovyan/')

In [4]:
from experiments.reporting.report_repository import ReportRepository
from experiments.slack_client import SlackNotifier
from neat.neat_logger import get_neat_logger
from neat.population_engine import EvolutionEngine
from neat.reporting.reports_pyneat import EvolutionReport
from neat.utils import timeit
import math
import random
from itertools import count
import numpy as np

from experiments.logger import logger
from experiments.slack_client import Notifier
from experiments.object_repository.object_repository import ObjectRepository
from experiments.reporting.report_repository import ReportPathFactory
from neat.configuration import get_configuration
from neat.evaluation.evaluation_engine import EvaluationStochasticEngine
from neat.evolution_operators.crossover import Crossover
from neat.evolution_operators.mutation import Mutation
from neat.genome import Genome
from neat.reporting.reports_pyneat import EvolutionReport
from neat.species import SpeciationEngine
from neat.stagnation import Stagnation
from neat.population_engine import PopulationEngine
from neat.utils import timeit
import neat
import tests
from config_files.configuration_utils import create_configuration
from neat.configuration import get_configuration, set_configuration
from neat.evaluation.evaluation_engine import get_dataset
from neat.evaluation.evaluate_simple import evaluate_genome, calculate_prediction_distribution
from neat.genome import Genome
from neat.evolution_operators.backprop_mutation import BackPropMutation
import os
from neat.loss.vi_loss import get_loss
import torch
import pandas as pd

from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, roc_curve

In [5]:
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
from neat.analysis.experiment_data import ExperimentDataNE, ExperimentDataNAS
from neat.analysis.uncertainty.predictive_distribution import PredictionDistributionEstimator
from neat.analysis.plotting.plot_losses import plot_loss_distribution
from neat.analysis.plotting.plot_classification_metrics import plot_metrics_by_quantile_several_executions
from neat.analysis.experiment_predictive_distributions import ExecutionsPredictionDistributions

In [7]:
LOGS_PATH = f'{os.getcwd()}/'
logger = get_neat_logger(path=LOGS_PATH)

In [9]:
config_versions = {'NE': {'project': 'neuro-evolution',
                         'algorithm_version': 'bayes-neat'},
                  'NAS': {'project': 'nas',
                         'algorithm_version': 'nas'}
                  }
datasets = ['iris', 'wine', 'breast_cancer', 'titanic', 'spambase', 'mnist_downsampled']
# datasets = [ 'wine']
# dataset = 'iris'

In [10]:
# df_versions = pd.DataFrame([['Bayesian-NEAT', 'NE', True, 'bayesian_neat_ft_21_iris'],
#                            ['NEAT', 'NE', False, 'neat_ft_21_iris'],
#                            ['Bayesian-MLP', 'NAS', True, 'bayesian_nas_final'],
# #                            ['NAS', False, 'standard_nas_final_with_reps']
#                            ['MLP', 'NAS', False, 'standard_nas_final']
#                            ], columns=['method', 'search', 'is_bayesian', 'correlation_id'])
versions_map = {}
for dataset in datasets:
    df_versions = pd.read_csv(f'./versions-2/{dataset}.csv')
    print(f'DATASET: {dataset}')
    print(df_versions)
    versions_map[dataset] = df_versions

DATASET: iris
          method search  is_bayesian                  correlation_id
0  Bayesian-NEAT     NE         True  bayesian_neat_ft_final_v1_iris
1           NEAT     NE        False           neat_ft_final_v1_iris
2   Bayesian-MLP    NAS         True           bayesian_nas_final_v2
3            MLP    NAS        False              standard_nas_final
DATASET: wine
          method search  is_bayesian                  correlation_id
0  Bayesian-NEAT     NE         True  bayesian_neat_ft_final_v1_wine
1           NEAT     NE        False           neat_ft_final_v1_wine
2   Bayesian-MLP    NAS         True           bayesian_nas_final_v2
3            MLP    NAS        False    standard_nas_final_v1_lenovo
DATASET: breast_cancer
          method search  is_bayesian                           correlation_id
0  Bayesian-NEAT     NE         True  bayesian_neat_ft_final_v1_breast_cancer
1           NEAT     NE        False           neat_ft_final_v1_breast_cancer
2   Bayesian-MLP    NAS  

In [10]:
# df_versions.to_csv(f'./versions/{dataset}.csv', index=False)

In [11]:
chunks_ne = []
chunks_nas = []
for dataset, df_versions in versions_map.items():
    print(dataset)
    for index, row in df_versions.iterrows():
        method = row.method
        version = row.search
        is_bayesian = row.is_bayesian
        correlation_id = row.correlation_id
        print(method, version, is_bayesian, correlation_id)
        if isinstance(correlation_id, float):
            continue
            
        n_samples = 1000 if is_bayesian else 1
        project = config_versions[version]['project']
        algorithm_version = config_versions[version]['algorithm_version']
        print(project, algorithm_version, n_samples)
        if version == 'NE':
            experiment_data = ExperimentDataNE(correlation_ids=[correlation_id], 
                                               dataset_name=dataset, 
                                               n_samples=n_samples,
                                               project = project, 
                                               algorithm_version = algorithm_version,
                                               keep_top=1.0, 
                                               filter_checkpoint_finish=True)\
                                .process_data()
            chunk_data_ne = experiment_data.get_experiment_data()
            # stay only with the best
            chunk_data_ne = chunk_data_ne.loc[chunk_data_ne['absolute_best'] == True]
            chunk_data_ne['version'] = version
            chunk_data_ne['is_bayesian'] = is_bayesian
            chunk_data_ne['method'] = method
            chunk_data_ne['dataset'] = dataset
            chunks_ne.append(chunk_data_ne)
        elif version == 'NAS':
            experiment_data_nas = ExperimentDataNAS(correlation_ids=[correlation_id], dataset_name=dataset, n_samples=n_samples,
                                                    project=config_versions[version]['project'], 
                                                    algorithm_version=config_versions[version]['algorithm_version'],
                                                    keep_top=1.0, filter_checkpoint_finish=True)
            experiment_data_nas.process_data()
            chunk_data_nas = experiment_data_nas.experiment_data
            chunk_data_nas['version'] = version
            chunk_data_nas['is_bayesian'] = is_bayesian
            chunk_data_nas['method'] = method
            chunk_data_nas['dataset'] = dataset
            chunks_nas.append(chunk_data_nas)
        else:
            raise ValueError()
data_ne = pd.concat(chunks_ne, sort=False)
data_nas = pd.concat(chunks_nas, sort=False)
# bayesian_neat_ft_final_v1_spambase

iris
Bayesian-NEAT NE True bayesian_neat_ft_final_v1_iris
neuro-evolution bayes-neat 1000
###########
CORRELATION ID: bayesian_neat_ft_final_v1_iris
5 executions
Train percentage: 0.75
Random state: 52
Sum Train: -41.176780700683594
Label Noise: 0.0
Sum of 10 first labels: 7
[2 1 0 1 1 0 1 0 0 1]
Training: 112. Testing: 38
Train percentage: 0.75
Random state: 52
Sum Train: -41.176780700683594
Label Noise: 0.0
Sum of 10 first labels: 7
[2 1 0 1 1 0 1 0 0 1]
Training: 112. Testing: 38
Train percentage: 0.75
Random state: 82
Sum Train: 0.3260765075683594
Label Noise: 0.0
Sum of 10 first labels: 13
[1 0 2 1 2 2 0 2 2 1]
Training: 112. Testing: 38
Train percentage: 0.75
Random state: 82
Sum Train: 0.3260765075683594
Label Noise: 0.0
Sum of 10 first labels: 13
[1 0 2 1 2 2 0 2 2 1]
Training: 112. Testing: 38


/opt/conda/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Train percentage: 0.75
Random state: 83
Sum Train: -16.439117431640625
Label Noise: 0.0
Sum of 10 first labels: 9
[2 0 1 1 1 0 1 0 2 1]
Training: 112. Testing: 38
Train percentage: 0.75
Random state: 83
Sum Train: -16.439117431640625
Label Noise: 0.0
Sum of 10 first labels: 9
[2 0 1 1 1 0 1 0 2 1]
Training: 112. Testing: 38
Train percentage: 0.75
Random state: 33
Sum Train: -32.15251159667969
Label Noise: 0.0
Sum of 10 first labels: 7
[1 0 1 1 1 0 0 1 0 2]
Training: 112. Testing: 38
Train percentage: 0.75
Random state: 33
Sum Train: -32.15251159667969
Label Noise: 0.0
Sum of 10 first labels: 7
[1 0 1 1 1 0 0 1 0 2]
Training: 112. Testing: 38


/opt/conda/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Train percentage: 0.75
Random state: 15
Sum Train: -0.34218311309814453
Label Noise: 0.0
Sum of 10 first labels: 9
[0 1 2 0 0 1 2 2 0 1]
Training: 112. Testing: 38
Train percentage: 0.75
Random state: 15
Sum Train: -0.34218311309814453
Label Noise: 0.0
Sum of 10 first labels: 9
[0 1 2 0 0 1 2 2 0 1]
Training: 112. Testing: 38
NEAT NE False neat_ft_final_v1_iris
neuro-evolution bayes-neat 1
###########
CORRELATION ID: neat_ft_final_v1_iris
5 executions


...

Sum Train: -6642.9501953125
Label Noise: 0.0
Sum of 10 first labels: 54
[8 4 4 9 7 4 8 1 3 6]
Training: 3750. Testing: 1250
Train percentage: 0.75
Random state: 49
Sum Train: -6642.9501953125
Label Noise: 0.0
Sum of 10 first labels: 54
[8 4 4 9 7 4 8 1 3 6]
Training: 3750. Testing: 1250
Train percentage: 0.75
Random state: 34
Sum Train: -6504.005859375
Label Noise: 0.0
Sum of 10 first labels: 45
[7 3 4 9 1 0 8 2 2 9]
Training: 3750. Testing: 1250


/opt/conda/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Train percentage: 0.75
Random state: 34
Sum Train: -6504.005859375
Label Noise: 0.0
Sum of 10 first labels: 45
[7 3 4 9 1 0 8 2 2 9]
Training: 3750. Testing: 1250
Train percentage: 0.75
Random state: 55
Sum Train: -6232.6494140625
Label Noise: 0.0
Sum of 10 first labels: 52
[6 7 6 6 5 2 0 9 9 2]
Training: 3750. Testing: 1250
Train percentage: 0.75
Random state: 55
Sum Train: -6232.6494140625
Label Noise: 0.0
Sum of 10 first labels: 52
[6 7 6 6 5 2 0 9 9 2]
Training: 3750. Testing: 1250
Train percentage: 0.75
Random state: 1
Sum Train: -6662.51611328125
Label Noise: 0.0
Sum of 10 first labels: 56
[8 2 3 9 6 8 4 5 8 3]
Training: 3750. Testing: 1250
Train percentage: 0.75
Random state: 1
Sum Train: -6662.51611328125
Label Noise: 0.0
Sum of 10 first labels: 56
[8 2 3 9 6 8 4 5 8 3]
Training: 3750. Testing: 1250
Train percentage: 0.75
Random state: 19
Sum Train: -7265.6689453125
Label Noise: 0.0
Sum of 10 first labels: 57
[2 7 8 3 9 8 4 6 7 3]
Training: 3750. Testing: 1250
Train percentage:

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/

NEAT NE False neat_ft_final_v1_mnist_downsampled
neuro-evolution bayes-neat 1
###########
CORRELATION ID: neat_ft_final_v1_mnist_downsampled
6 executions
Train percentage: 0.75
Random state: 95
Sum Train: -6777.90380859375
Label Noise: 0.0
Sum of 10 first labels: 45
[2 9 8 0 5 3 7 7 0 4]
Training: 3750. Testing: 1250
Train percentage: 0.75
Random state: 95
Sum Train: -6777.90380859375
Label Noise: 0.0
Sum of 10 first labels: 45
[2 9 8 0 5 3 7 7 0 4]
Training: 3750. Testing: 1250
Train percentage: 0.75
Random state: 71
Sum Train: -6930.7744140625
Label Noise: 0.0
Sum of 10 first labels: 42
[2 8 9 1 5 1 6 4 3 3]
Training: 3750. Testing: 1250
Train percentage: 0.75
Random state: 6
Sum Train: -6952.056640625
Label Noise: 0.0
Sum of 10 first labels: 38
[1 5 6 1 2 3 6 2 3 9]
Training: 3750. Testing: 1250
Train percentage: 0.75
Random state: 6
Sum Train: -6952.056640625
Label Noise: 0.0
Sum of 10 first labels: 38
[1 5 6 1 2 3 6 2 3 9]
Training: 3750. Testing: 1250
Train percentage: 0.75
Rando

In [12]:
display(data_ne.head(1))
display(data_nas.head(1))

,correlation_id,execution_id,train_percentage,noise,label_noise,is_bayesian,beta,loss_training,loss_testing,duration,...,mean_genome_std,accuracy,precision,recall,f1,ece,absolute_best,version,method,dataset
0,bayesian_neat_ft_final_v1_iris,0f5a1f1c-048b-4e92-bc72-81b00fa4041d,0.75,0.0,0.0,True,0.0001,0.59272,0.848406,0:08:55.735616,...,0.955271,97.368421,0.975232,0.973684,0.973557,0.210877,True,NE,Bayesian-NEAT,iris


,correlation_id,execution_id,train_percentage,noise,label_noise,is_bayesian,beta,duration,end_condition,n_parameters,...,n_connections,n_layers,f1,recall,precision,accuracy,ece,version,method,dataset
0,bayesian_nas_final_v2,019c4bee-36ff-431a-8079-c2d5abd674ee,0.75,0.0,0.4,True,0.0001,0:15:13.648590,normal,246,...,105,2,0.733791,0.736842,0.868421,73.684211,0.142737,NAS,Bayesian-MLP,iris


In [13]:
from scipy import stats
def filter_data(data, filter_noisy=False, z_score=3):
#     data = data.loc[data['f1'] >= filter_threshold]
    chunks = []
    for _, df in data.groupby(['method', 'label_noise']):
#         df[(np.abs(stats.zscore(df['f1'])) < 3).all(axis=1)]
        df = df.loc[np.abs(stats.zscore(df['f1'])) < z_score]
        chunks.append(df)
    data = pd.concat(chunks)
    if filter_noisy:
        data = data.loc[(data['noise'] == 0) &
               (data['label_noise'] == 0)]
    
    
    return data

Filter noisy executions

In [14]:
display(data_ne.shape)
display(data_nas.shape)

(60, 25)

(820, 21)

In [15]:
data_ne = data_ne.loc[(data_ne['noise'] == 0) &
                     (data_ne['label_noise'] == 0)]
data_nas = data_nas.loc[(data_nas['noise'] == 0) &
                     (data_nas['label_noise'] == 0)]

In [16]:
display(data_ne.shape)
display(data_nas.shape)

(60, 25)

(100, 21)

In [17]:
version = 12 # new b-mlp
for dataset, data_dataset in data_ne.groupby('dataset'):
    data_dataset.to_csv(f'data/data_experiment_2_ne_{dataset}_v{version}.csv', index=False)

for dataset, data_dataset in data_nas.groupby('dataset'):
    data_dataset.to_csv(f'data/data_experiment_2_nas_{dataset}_v{version}.csv', index=False)
